## 数据清洗及可视化

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
from typing import List, Tuple, Dict

# -------------------------- 1. 全局配置--------------------------
# 字体与显示配置（解决中文乱码）
plt.rcParams['font.sans-serif'] = ['Hiragino Sans GB', 'STHeiti', 'Arial']
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')
warnings.filterwarnings('ignore', category=FutureWarning)

os.makedirs("visualization_results", exist_ok=True)

# 固定配置
POWER_CONFIGS = [
    ("Zone 1 Power Consumption", "区域1电力消耗 (kW)", "#45B7D1"),
    ("Zone 2  Power Consumption", "区域2电力消耗 (kW)", "#45B7D1"),  # 列名含2个空格
    ("Zone 3  Power Consumption", "区域3电力消耗 (kW)", "#45B7D1")   # 列名含2个空格
]
FEATURE_COLS = [
    'Temperature', 'Humidity', 'Wind Speed', 
    'general diffuse flows', 'diffuse flows'
]
UNIT_MAP = {
    'Temperature': '°C',
    'Humidity': '%',
    'Wind Speed': 'm/s',
    'general diffuse flows': 'W/m²',
    'diffuse flows': 'W/m²'
}

# -------------------------- 2. 类定义 --------------------------
class PowerConsumptionAnalyzer:
    def __init__(self, data_path: str = "consumption.csv"):
        self.data_path = data_path
        self.df_raw = None  # 原始数据
        self.df_processed = None  # 日期索引处理后的数据
        self.df_daily_mean = None  # 日均值数据
        self.feature_data = None  # 特征数据（用于后续算法）

    def load_data(self) -> None:
        try:
            # 检查文件是否存在
            if not os.path.exists(self.data_path):
                raise FileNotFoundError(f"数据文件不存在：{self.data_path}")
            
            # 读取数据
            self.df_raw = pd.read_csv(self.data_path)
            print(f"✅ 成功加载原始数据，形状：{self.df_raw.shape}")
            
            # 检查必要列是否存在
            required_cols = ["DateTime"] + [cfg[0] for cfg in POWER_CONFIGS] + FEATURE_COLS
            missing_cols = [col for col in required_cols if col not in self.df_raw.columns]
            if missing_cols:
                raise ValueError(f"数据缺失必要列：{missing_cols}")
        
        except Exception as e:
            raise RuntimeError(f"❌ 数据加载失败：{str(e)}")

    def process_data(self) -> None:
        if self.df_raw is None:
            raise ValueError("请先调用 load_data() 加载数据")
        
        try:
            # 日期转换与索引设置
            self.df_processed = self.df_raw.copy()
            self.df_processed["DateTime"] = pd.to_datetime(self.df_processed["DateTime"], errors='coerce')
            
            # 检查日期转换有效性
            invalid_date_count = self.df_processed["DateTime"].isnull().sum()
            if invalid_date_count > 0:
                raise ValueError(f"日期转换失败，共{invalid_date_count}条无效日期数据")
            
            self.df_processed.set_index("DateTime", inplace=True)
            
            # 日均值重采样
            self.df_daily_mean = self.df_processed.resample('D').mean()
            
            # 检查重采样后数据有效性
            if self.df_daily_mean.empty:
                raise ValueError("日均值重采样后数据为空，请检查原始数据时间范围")
            
            # 处理特征列缺失值（用户指定逻辑）
            self.feature_data = self.df_daily_mean[FEATURE_COLS].fillna(method='ffill').fillna(method='bfill')
            
            print(f"✅ 数据预处理完成")
            print(f"  - 处理后数据形状：{self.df_processed.shape}")
            print(f"  - 日均值数据形状：{self.df_daily_mean.shape}")
            print(f"  - 特征数据形状：{self.feature_data.shape}")
        
        except Exception as e:
            raise RuntimeError(f"❌ 数据预处理失败：{str(e)}")

    def plot_power_time_series(self) -> None:
        """绘制3个区域电力消耗日均值时序图"""
        if self.df_daily_mean is None:
            raise ValueError("请先调用 process_data() 处理数据")
        
        # 创建3行1列子图
        fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(14, 12), sharex=True)
        
        # 遍历每个区域绘制
        for idx, (col_name, chinese_name, color) in enumerate(POWER_CONFIGS):
            ax = axes[idx]
            # 绘制时序曲线（含圆点优化）
            ax.plot(
                self.df_daily_mean.index, self.df_daily_mean[col_name],
                color=color, linewidth=2.5, alpha=0.9,
                markersize=5, markerfacecolor='white', markeredgewidth=1.5
            )
            
            # 图表样式配置
            ax.set_title(f"{chinese_name}（日均值时间序列）", fontsize=15, fontweight="bold", pad=15)
            ax.set_ylabel(chinese_name.split(" ")[-1], fontsize=11, fontweight="medium")
            ax.tick_params(axis='y', labelsize=10)
            ax.grid(True, alpha=0.3, linestyle='--')
            
            # 调整y轴范围（上下留空5%）
            y_min = self.df_daily_mean[col_name].min() * 0.95
            y_max = self.df_daily_mean[col_name].max() * 1.05
            ax.set_ylim(y_min, y_max)
            
            # 最后一个子图显示x轴标签
            if idx == 2:
                ax.set_xlabel("日期", fontsize=12, fontweight="medium")
                ax.tick_params(axis='x', rotation=45, labelsize=10)
        
        # 调整x轴范围
        x_min = self.df_daily_mean.index.min() - pd.Timedelta(days=1)
        x_max = self.df_daily_mean.index.max() + pd.Timedelta(days=1)
        for ax in axes:
            ax.set_xlim(x_min, x_max)
        
        # 调整子图间距
        plt.tight_layout(pad=3.0)
        plt.savefig("visualization_results/power_daily_mean_series.png", dpi=300, bbox_inches='tight')
        plt.show()
        print("✅ 电力消耗时序图已保存至 visualization_results/")

    def plot_feature_time_series(self) -> None:
        """绘制5个环境特征时序图"""
        if self.feature_data is None:
            raise ValueError("请先调用 process_data() 处理数据")
        
        # 创建3行2列子图
        fig, axes = plt.subplots(3, 2, figsize=(16, 10), sharex=True)
        fig.suptitle('环境特征时序变化趋势', fontsize=18, fontweight='bold', y=0.98)
        
        # 子图位置映射（5个特征对应前5个位置）
        subplot_pos = [(0,0), (0,1), (1,0), (1,1), (2,0)]
        
        # 逐个绘制特征
        for idx, (feat, pos) in enumerate(zip(FEATURE_COLS, subplot_pos)):
            ax = axes[pos]
            # 绘制时序曲线（默认颜色）
            ax.plot(
                self.feature_data.index, self.feature_data[feat],
                linewidth=1.8, alpha=0.8, label=feat
            )
            # 添加均值线
            mean_val = self.feature_data[feat].mean()
            ax.axhline(
                y=mean_val, color='black', linestyle='--', linewidth=1.2, alpha=0.7,
                label=f'均值：{mean_val:.2f} {UNIT_MAP.get(feat, "")}'
            )
            
            # 样式配置
            ax.set_title(feat, fontsize=12, fontweight='medium', pad=8)
            ax.set_ylabel(f'数值（{UNIT_MAP.get(feat, "无单位")}）', fontsize=10)
            ax.grid(True, linestyle='--', alpha=0.3)
            ax.legend(loc='upper right', fontsize=9, framealpha=0.9)
            ax.tick_params(axis='y', labelsize=9)
        
        # 隐藏最后一个空子图
        axes[2, 1].axis('off')
        
        # 配置x轴标签
        axes[2, 0].set_xlabel('时间', fontsize=12, fontweight='medium')
        axes[2, 0].tick_params(axis='x', rotation=45, labelsize=9)
        
        # 调整间距（预留标题空间）
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        # 保存图片
        plt.savefig("visualization_results/feature_time_series.png", dpi=300, bbox_inches='tight')
        plt.show()
        print("✅ 特征时序图已保存至 visualization_results/")

    def get_processed_data(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """获取处理后的数据"""
        if self.df_daily_mean is None or self.feature_data is None:
            raise ValueError("请先调用 process_data() 处理数据")
        return self.df_daily_mean, self.feature_data

# -------------------------- 3. 辅助函数（满足函数调用要求） --------------------------
def validate_data_quality(df: pd.DataFrame, data_name: str) -> bool:
    """验证数据质量（独立函数，体现函数调用）"""
    # 检查数据非空
    if df.empty:
        print(f"❌ {data_name} 数据为空")
        return False
    # 检查特征列缺失值
    missing_ratio = df[FEATURE_COLS].isnull().sum() / len(df) * 100
    high_missing_cols = missing_ratio[missing_ratio > 30].index.tolist()
    if high_missing_cols:
        print(f"⚠️ {data_name} 以下特征缺失率超过30%：{high_missing_cols}")
    return True

# -------------------------- 4. 主函数 --------------------------
def main():
    
    try:
        # 1. 初始化分析器
        analyzer = PowerConsumptionAnalyzer(data_path="consumption.csv")
        
        # 2. 加载数据
        analyzer.load_data()
        
        # 3. 数据预处理
        analyzer.process_data()
        
        # 4. 验证数据质量
        df_daily_mean, feature_data = analyzer.get_processed_data()
        if validate_data_quality(df_daily_mean, "日均值数据") and validate_data_quality(feature_data, "特征数据"):
            print("✅ 数据质量验证通过")
        
        # 5. 绘制可视化图表
        print("\n📊 开始绘制电力消耗时序图...")
        analyzer.plot_power_time_series()
        
        print("\n📊 开始绘制特征时序图...")
        analyzer.plot_feature_time_series()
        
        print("\n" + "="*60)
        print("🎉 数据预处理与可视化流程完成！")
        print("="*60)
        print("输出文件清单：")
        print("1. 电力消耗时序图：visualization_results/power_daily_mean_series.png")
        print("2. 特征时序图：visualization_results/feature_time_series.png")
    
    except Exception as e:
        print(f"\n❌ 流程执行失败：{str(e)}")
        raise

if __name__ == "__main__":
    main()


KeyboardInterrupt



## ARIMA模型

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

# 基础配置
os.makedirs("results", exist_ok=True)
plt.rcParams['font.sans-serif'] = ['Hiragino Sans GB', 'STHeiti', 'Arial']
plt.rcParams['axes.unicode_minus'] = False

# 评估指标函数
def calculate_mape(y_true, y_pred, eps=1e-8):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_true_safe = np.where(y_true == 0, eps, y_true)
    mape = np.mean(np.abs((y_true_safe - y_pred) / y_true_safe)) * 100
    return round(mape, 2)

def calculate_metrics(y_true, y_pred):
    mae = np.mean(np.abs(y_true - y_pred))
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    mape = calculate_mape(y_true, y_pred)
    return round(mae, 2), round(rmse, 2), mape

# 核心类定义
class ARIMATimeSeriesAnalyzer:
    def __init__(self, data_path, target_col):
        self.data_path = data_path
        self.target_col = target_col
        self.df = None
        self.df_daily_mean = None
        self.target_series = None
        self.train_series = None
        self.test_series = None
        self.best_order = None
        self.model = None
        self.model_result = None

    def load_and_preprocess(self):
        """加载数据并预处理（日均值重采样）"""
        try:
            if not os.path.exists(self.data_path):
                raise FileNotFoundError(f"数据文件不存在：{self.data_path}")
            
            self.df = pd.read_csv(self.data_path, parse_dates=['date'], index_col='date')
            
            # 检查目标列是否存在
            if self.target_col not in self.df.columns:
                raise ValueError(f"数据中缺失目标列：{self.target_col}")
            
            # 日均值重采样
            self.df_daily_mean = self.df.resample('D').mean().dropna()
            self.target_series = self.df_daily_mean[self.target_col].dropna()
            
            # 检查数据有效性
            if len(self.target_series) < 30:
                raise ValueError(f"有效数据不足30条，当前仅{len(self.target_series)}条")
            
            print(f"数据加载完成：序列长度={len(self.target_series)}, 数值范围=[{self.target_series.min():.2f}, {self.target_series.max():.2f}] kW")
            return self
        except Exception as e:
            raise RuntimeError(f"数据加载失败：{str(e)}")

    def split_train_test(self, train_ratio=0.8):
        """时间顺序划分训练集/测试集"""
        train_size = int(train_ratio * len(self.target_series))
        self.train_series = self.target_series.iloc[:train_size]
        self.test_series = self.target_series.iloc[train_size:]
        print(f"训练集：{len(self.train_series)}条数据 | 测试集：{len(self.test_series)}条数据")
        return self

    def find_min_diff_order(self, max_diff=3):
        """差分平稳化，寻找最小d阶"""
        print("\n" + "="*60)
        print("差分平稳化过程")
        print("="*60)
        
        for d in range(max_diff + 1):
            diff_series = self.train_series.diff(d).dropna() if d > 0 else self.train_series.copy()
            if len(diff_series) < 10:
                continue
            
            adf_result = adfuller(diff_series)
            p_value = adf_result[1]
            print(f"{d}阶差分后：数据量={len(diff_series)} | p值={p_value:.4f}")
            
            if p_value < 0.05:
                print(f"→ 平稳（p<0.05），最小差分阶数d={d}")
                return d
        
        print(f"警告：{max_diff}阶差分后仍非平稳，使用d={max_diff}")
        return max_diff

    def auto_select_pq(self, d, max_p=3, max_q=3):
        """自动选择p、q阶数（基于AIC准则）"""
        best_aic = float('inf')
        best_pq = (0, 0)
        
        for p in range(max_p + 1):
            for q in range(max_q + 1):
                try:
                    temp_model = ARIMA(self.train_series, order=(p, d, q))
                    temp_result = temp_model.fit()
                    if temp_result.aic < best_aic:
                        best_aic = temp_result.aic
                        best_pq = (p, q)
                except:
                    continue
        
        p, q = best_pq
        self.best_order = (p, d, q)
        print(f"\n自动定阶结果：p={p}, d={d}, q={q} | 最小AIC={best_aic:.2f}")
        print(f"最终ARIMA模型：ARIMA{self.best_order}")
        return self

    def train_model(self):
        """训练ARIMA模型"""
        print("\n" + "="*60)
        print(f"ARIMA{self.best_order} 模型训练")
        print("="*60)
        
        try:
            self.model = ARIMA(self.train_series, order=self.best_order)
            self.model_result = self.model.fit()
            
            # 输出核心参数
            print("\n模型核心参数：")
            params_df = pd.DataFrame({
                '参数名称': self.model_result.params.index[:5],  # 只显示前5个核心参数
                '参数值': self.model_result.params.values[:5].round(4),
                'p值': self.model_result.pvalues.values[:5].round(4)
            })
            print(params_df)
            return self
        except Exception as e:
            raise RuntimeError(f"模型训练失败：{str(e)}")

    def predict(self):
        """训练集拟合+测试集预测"""
        # 训练集内拟合
        self.in_sample_pred = self.model_result.predict(
            start=self.train_series.index[1],
            end=self.train_series.index[-1],
            typ='levels'
        )
        self.in_sample_ci = self.model_result.get_prediction(
            start=self.train_series.index[1],
            end=self.train_series.index[-1]
        ).conf_int()
        
        # 测试集预测
        test_forecast = self.model_result.get_forecast(steps=len(self.test_series))
        self.test_pred = test_forecast.predicted_mean
        self.test_ci = test_forecast.conf_int()
        
        return self

    def evaluate(self):
        """计算训练集和测试集误差"""
        print("\n" + "="*60)
        print("误差评估汇总")
        print("="*60)
        
        # 训练集评估
        train_true = self.train_series.loc[self.in_sample_pred.index]
        self.train_mae, self.train_rmse, self.train_mape = calculate_metrics(
            train_true.values, self.in_sample_pred.values
        )
        print(f"【训练集】")
        print(f"有效拟合点数：{len(self.in_sample_pred)}")
        print(f"MAE：{self.train_mae} kW | RMSE：{self.train_rmse} kW | MAPE：{self.train_mape}%")
        
        # 测试集评估
        self.test_mae, self.test_rmse, self.test_mape = calculate_metrics(
            self.test_series.values, self.test_pred.values
        )
        print(f"【测试集】")
        print(f"预测点数：{len(self.test_pred)}")
        print(f"MAE：{self.test_mae} kW | RMSE：{self.test_rmse} kW | MAPE：{self.test_mape}%")
        return self

    def plot_result(self):
        """绘制训练集拟合+测试集预测结果"""
        fig, ax = plt.subplots(figsize=(16, 8))
        
        # 绘制真实值
        ax.plot(self.target_series.index, self.target_series, color='#FF6B6B', 
                linewidth=2.5, label='真实值（区域3电力消耗）', alpha=0.8, marker='o', markersize=4)
        
        # 绘制训练集拟合
        ax.plot(self.in_sample_pred.index, self.in_sample_pred, color='#45B7D1', 
                linewidth=3, label=f'ARIMA{self.best_order} 训练集拟合')
        ax.fill_between(self.in_sample_ci.index, 
                        self.in_sample_ci.iloc[:, 0], self.in_sample_ci.iloc[:, 1],
                        color='#45B7D1', alpha=0.25, label='训练集95%置信区间')
        
        # 绘制测试集预测
        ax.plot(self.test_series.index, self.test_pred, linewidth=3, color='#96CEB4',
                label=f'ARIMA{self.best_order} 测试集预测')
        ax.fill_between(self.test_ci.index, 
                        self.test_ci.iloc[:, 0], self.test_ci.iloc[:, 1],
                        color='#96CEB4', alpha=0.25, label='测试集95%置信区间')
        
        # 分割线
        ax.axvline(x=self.train_series.index[-1], color='black', linestyle=':', 
                   linewidth=2, label='训练/测试集分割线')
        
        # 图表样式
        ax.set_title('区域3电力消耗 - ARIMA模型 训练集拟合+测试集预测', 
                     fontsize=18, fontweight='bold', pad=20)
        ax.set_xlabel('日期', fontsize=14, fontweight='medium')
        ax.set_ylabel('电力消耗 (kW)', fontsize=14, fontweight='medium')
        ax.tick_params(axis='x', rotation=45, labelsize=12)
        ax.tick_params(axis='y', labelsize=12)
        ax.grid(True, alpha=0.3, linestyle='--')
        
        # 坐标轴范围优化
        y_min = self.target_series.min() * 0.95
        y_max = self.target_series.max() * 1.05
        x_min = self.target_series.index.min() - pd.Timedelta(days=1)
        x_max = self.target_series.index.max() + pd.Timedelta(days=1)
        ax.set_ylim(y_min, y_max)
        ax.set_xlim(x_min, x_max)
        
        ax.legend(loc='upper right', fontsize=12, framealpha=0.9)
        plt.tight_layout()
        plt.savefig("results/arima_train_test_fitting.png", dpi=300, bbox_inches="tight")
        plt.show()
        print("\n可视化图表已保存至 results/arima_train_test_fitting.png")
        return self

# 主函数
def main():
    print("="*60)
    print("区域3电力消耗ARIMA时间序列预测")
    print("="*60)
    
    # 配置参数
    DATA_PATH = "consumption.csv"
    TARGET_COL = "Zone 3  Power Consumption"  
    
    try:
        # 串联整个流程
        analyzer = ARIMATimeSeriesAnalyzer(DATA_PATH, TARGET_COL)
        analyzer.load_and_preprocess()\
               .split_train_test()
               
        d = analyzer.find_min_diff_order()
        analyzer.auto_select_pq(d)\
               .train_model()\
               .predict()\
               .evaluate()\
               .plot_result()
        
        print("\n" + "="*60)
        print("项目运行完成！")
        print("="*60)
        print("输出文件清单：")
        print("- 预测可视化图：results/arima_train_test_fitting.png")
        
    except Exception as e:
        print(f"\n运行失败：{str(e)}")
        raise

if __name__ == "__main__":
    main()


KeyboardInterrupt



## transformer模型

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import random
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

os.makedirs("best_model", exist_ok=True)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 评估指标函数
def calculate_mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def calculate_mape(y_true, y_pred, epsilon=1e-8):
    y_true = np.array(y_true) + epsilon
    y_pred = np.array(y_pred)
    return round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 4)

# 数据集类
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_len, pred_len=1):
        self.data = data
        self.seq_len = seq_len
        self.pred_len = pred_len
        
        if len(data) < seq_len + pred_len:
            raise ValueError(f"数据长度不足（需≥{seq_len+pred_len}，实际{len(data)}）")
        self.length = len(data) - seq_len - pred_len + 1

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        if idx < 0 or idx >= self.length:
            raise IndexError(f"索引{idx}超出范围（0-{self.length-1}）")
        x = self.data[idx:idx+self.seq_len]
        y = self.data[idx+self.seq_len:idx+self.seq_len+self.pred_len]
        return torch.tensor(x, dtype=torch.float32).unsqueeze(-1), torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

# Transformer模型类
class TransformerModel(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_heads=4, num_transformer_layers=2, pred_len=1):
        super().__init__()
        self.pred_len = pred_len
        
        if hidden_dim % num_heads != 0:
            raise ValueError(f"hidden_dim({hidden_dim})必须能被num_heads({num_heads})整除")
        
        self.input_proj = nn.Linear(input_dim, hidden_dim)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=hidden_dim, nhead=num_heads, dim_feedforward=hidden_dim * 4,
                batch_first=True, norm_first=True, dropout=0.1
            ),
            num_layers=num_transformer_layers
        )
        self.fc = nn.Linear(hidden_dim, pred_len)

    def forward(self, x):
        x_proj = self.input_proj(x)
        x_transformer = self.transformer_encoder(x_proj)
        out = self.fc(x_transformer[:, -1, :])
        return out.unsqueeze(-1)

# 数据加载函数
def load_data(data_path):
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"数据文件不存在：{data_path}")
    
    df = pd.read_csv(data_path, parse_dates=['DateTime'], index_col='DateTime')
    target_col = 'Zone 3  Power Consumption'
    
    if target_col not in df.columns:
        raise ValueError(f"数据缺失目标列：{target_col}")
    
    df_daily_mean = df.resample('D').mean().dropna()
    temp_series1 = df_daily_mean[target_col].copy()
    print(f"数据加载完成：序列长度={len(temp_series1)}")
    return temp_series1

# 数据预处理函数
def preprocess_data(series):
    # 一阶差分平稳化
    diff_series = series.diff(1).dropna()
    # 标准化
    scaler = StandardScaler()
    diff_series_scaled = scaler.fit_transform(diff_series.values.reshape(-1, 1)).flatten()
    # 划分训练/测试集
    train_size = int(0.8 * len(diff_series_scaled))
    train_data = diff_series_scaled[:train_size]
    test_data = diff_series_scaled[train_size:]
    
    print(f"数据预处理完成：训练集{len(train_data)}条 | 测试集{len(test_data)}条")
    return train_data, test_data, scaler, diff_series.index

# 预测与逆变换函数
def predict(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            pred = model(x)
            preds.extend(pred.cpu().numpy().flatten())
            trues.extend(y.cpu().numpy().flatten())
    return np.array(preds), np.array(trues)

def inverse_transform(pred_scaled, scaler, series, seq_len, train_size, is_train=True):
    pred_diff = scaler.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
    start_idx = seq_len if is_train else train_size + seq_len
    original_vals = series.iloc[start_idx-1 : start_idx-1 + len(pred_diff)].values
    return original_vals + pred_diff

# 训练评估函数
def train_eval(params, train_data, test_data, scaler, series, PRED_LEN, EPOCHS_BASE):
    try:
        train_dataset = TimeSeriesDataset(train_data, params['SEQ_LEN'], PRED_LEN)
        test_dataset = TimeSeriesDataset(test_data, params['SEQ_LEN'], PRED_LEN)
    except Exception as e:
        print(f"数据集构建失败：{e}")
        return float('inf'), float('inf'), None
    
    if len(train_dataset) < 10 or len(test_dataset) < 5:
        return float('inf'), float('inf'), None
    
    train_loader = DataLoader(train_dataset, batch_size=params['BATCH_SIZE'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=params['BATCH_SIZE'], shuffle=False)
    
    model = TransformerModel(
        input_dim=1,
        hidden_dim=params['hidden_dim'],
        num_heads=params['num_heads'],
        num_transformer_layers=params['num_transformer_layers'],
        pred_len=PRED_LEN
    ).to(DEVICE)
    
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=params['LEARNING_RATE'], weight_decay=1e-5)
    
    # 训练
    for _ in range(EPOCHS_BASE):
        model.train()
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
            y_pred = model(x_batch)
            loss = criterion(y_pred, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
    
    # 测试集预测与评估
    test_pred_scaled, test_true_scaled = predict(model, test_loader)
    test_pred = inverse_transform(test_pred_scaled, scaler, series, params['SEQ_LEN'], len(train_data), is_train=False)
    test_true = inverse_transform(test_true_scaled, scaler, series, params['SEQ_LEN'], len(train_data), is_train=False)
    
    test_mae = calculate_mae(test_true, test_pred)
    test_mape = calculate_mape(test_true, test_pred)
    return test_mae, test_mape, model

# 可视化函数
def visualize_results(best_params, best_model, train_data, test_data, scaler, series, PRED_LEN):
    # 构建最优模型数据集
    best_train_dataset = TimeSeriesDataset(train_data, best_params['SEQ_LEN'], PRED_LEN)
    best_test_dataset = TimeSeriesDataset(test_data, best_params['SEQ_LEN'], PRED_LEN)
    best_train_loader = DataLoader(best_train_dataset, batch_size=best_params['BATCH_SIZE'], shuffle=False)
    best_test_loader = DataLoader(best_test_dataset, batch_size=best_params['BATCH_SIZE'], shuffle=False)
    
    # 预测
    train_pred_scaled, train_true_scaled = predict(best_model, best_train_loader)
    test_pred_scaled, test_true_scaled = predict(best_model, best_test_loader)
    
    # 逆变换
    train_pred = inverse_transform(train_pred_scaled, scaler, series, best_params['SEQ_LEN'], len(train_data), is_train=True)
    train_true = inverse_transform(train_true_scaled, scaler, series, best_params['SEQ_LEN'], len(train_data), is_train=True)
    test_pred = inverse_transform(test_pred_scaled, scaler, series, best_params['SEQ_LEN'], len(train_data), is_train=False)
    test_true = inverse_transform(test_true_scaled, scaler, series, best_params['SEQ_LEN'], len(train_data), is_train=False)
    
    # 计算指标
    train_mae = calculate_mae(train_true, train_pred)
    train_mape = calculate_mape(train_true, train_pred)
    
    # 绘制图表
    fig, ax = plt.subplots(figsize=(18, 8))
    train_steps = range(len(train_true))
    test_steps = range(len(train_true), len(train_true) + len(test_true))
    
    ax.plot(train_steps, train_true, color='#2E86AB', linewidth=2.0, 
            label=f'训练集真实值 (MAE={train_mae:.2f} kW)', alpha=0.8)
    ax.plot(train_steps, train_pred, color='#F2A154', linewidth=1.8, label='训练集预测值', linestyle='-', alpha=0.9)
    ax.plot(test_steps, test_true, color='#2E86AB', linewidth=2.5, 
            label=f'测试集真实值 (MAE={best_mae:.2f} kW)', alpha=0.8)
    ax.plot(test_steps, test_pred, color='#E63946', linewidth=2.2, label='测试集预测值', linestyle='--', alpha=0.9)
    ax.axvline(x=len(train_true)-1, color='black', linestyle=':', linewidth=2.5, label='训练/测试集分割线')
    
    ax.set_title('区域3电力消耗 - 最优纯Transformer模型 训练集+测试集预测结果', fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('时间步（按日期顺序）', fontsize=14), ax.set_ylabel('电力消耗 (kW)', fontsize=14)
    ax.tick_params(axis='x', labelsize=12), ax.tick_params(axis='y', labelsize=12)
    ax.grid(True, alpha=0.3, linestyle='--'), ax.legend(loc='upper right', fontsize=12)
    
    plt.tight_layout()
    plt.savefig("best_model/transformer_train_test_prediction.png", bbox_inches='tight', dpi=300)
    plt.show()
    
    return train_mae, train_mape

# 主函数
def main():
    print("="*60)
    print("区域3电力消耗预测（纯Transformer模型）")
    print("="*60)
    
    # 配置参数
    DATA_PATH = "consumption.csv"
    PRED_LEN = 1
    EPOCHS_BASE = 40
    
    # 超参数搜索空间
    param_grid = {
        'SEQ_LEN': [28],
        'hidden_dim': [32, 64, 128, 256],
        'num_heads': [2, 4, 8, 16],
        'num_transformer_layers': [1, 2, 3, 4],
        'BATCH_SIZE': [8, 16, 32, 64],
        'LEARNING_RATE': [5e-5, 1e-4, 5e-4, 1e-3]
    }
    
    # 生成20组有效参数
    n_trials = 20
    random_params = []
    while len(random_params) < n_trials:
        params = {k: random.choice(v) for k, v in param_grid.items()}
        if params['hidden_dim'] % params['num_heads'] == 0:
            random_params.append(params)
    
    try:
        # 数据加载与预处理
        series = load_data(DATA_PATH)
        train_data, test_data, scaler, diff_index = preprocess_data(series)
        
        # 超参数搜索
        print("\n===== 开始20组超参数随机搜索 =====")
        best_mae = float('inf')
        best_params = None
        best_model = None
        
        for i, params in enumerate(tqdm(random_params, desc="调参进度")):
            try:
                test_mae, test_mape, model = train_eval(params, train_data, test_data, scaler, series, PRED_LEN, EPOCHS_BASE)
                print(f"第{i+1}组：MAE={test_mae:.6f} | MAPE={test_mape:.4f}%")
                
                if test_mae < best_mae:
                    best_mae = test_mae
                    best_mape = test_mape
                    best_params = params
                    best_model = model
            
            except Exception as e:
                print(f"第{i+1}组参数训练失败：{str(e)}")
                continue
        
        # 最优模型结果输出
        print("\n" + "="*100)
        print("最优纯Transformer模型结果汇总")
        print("="*100)
        print(f"最优参数组合：")
        for k, v in best_params.items():
            print(f"  - {k}: {v}")
        print(f"\n测试集MAE：{best_mae:.6f} kW | MAPE：{best_mape:.4f}%")
        
        # 可视化与保存
        train_mae, train_mape = visualize_results(best_params, best_model, train_data, test_data, scaler, series, PRED_LEN)
        
        # 保存模型
        torch.save({
            'model_state_dict': best_model.state_dict(),
            'params': best_params,
            'scaler': scaler,
            'train_mae': train_mae,
            'train_mape': train_mape,
            'test_mae': best_mae,
            'test_mape': best_mape
        }, "best_model/best_transformer_model.pth")
        
        print(f"\n" + "="*100)
        print("可视化与模型保存完成")
        print("="*100)
        print(f"📈 可视化图已保存至：best_model/transformer_train_test_prediction.png")
        print(f"💾 模型已保存至：best_model/best_transformer_model.pth")
        print(f"\n最终统计指标：")
        print(f"  - 训练集 MAE：{train_mae:.6f} kW | MAPE：{train_mape:.4f}%")
        print(f"  - 测试集 MAE：{best_mae:.6f} kW | MAPE：{best_mape:.4f}%")
    
    except Exception as e:
        print(f"\n运行失败：{str(e)}")
        raise

if __name__ == "__main__":
    main()


KeyboardInterrupt



## 单指标Transformer+cnn模型

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import random
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

# 基础配置
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs("best_model", exist_ok=True)
os.makedirs("eda_plots", exist_ok=True)

# 评估指标计算
def calculate_mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def calculate_mape(y_true, y_pred, epsilon=1e-8):
    y_true = np.array(y_true) + epsilon
    y_pred = np.array(y_pred)
    return round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 4)

# -------------------------- 核心类定义 --------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_len, pred_len=1):
        self.data = data
        self.seq_len = seq_len
        self.pred_len = pred_len
        
        if len(data) < seq_len + pred_len:
            raise ValueError(f"数据长度({len(data)})不足，需至少{seq_len + pred_len}个样本")
        self.length = len(data) - seq_len - pred_len + 1

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        if idx < 0 or idx >= self.length:
            raise IndexError(f"索引{idx}超出范围（0-{self.length-1}）")
        x = self.data[idx:idx+self.seq_len]
        y = self.data[idx+self.seq_len:idx+self.seq_len+self.pred_len]
        return torch.tensor(x, dtype=torch.float32).unsqueeze(-1), torch.tensor(y, dtype=torch.float32).unsqueeze(-1)

class CNNTransformerModel(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_heads=4, num_transformer_layers=2, kernel_size=3, pred_len=1):
        super().__init__()
        if hidden_dim % num_heads != 0:
            raise ValueError(f"hidden_dim({hidden_dim})必须能被num_heads({num_heads})整除")
        
        self.cnn = nn.Sequential(
            nn.Conv1d(input_dim, hidden_dim, kernel_size, padding=kernel_size-1, dilation=1),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size, padding=2*(kernel_size-1), dilation=2),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=hidden_dim, nhead=num_heads, dim_feedforward=hidden_dim*4,
                batch_first=True, norm_first=True, dropout=0.1
            ),
            num_layers=num_transformer_layers
        )
        self.fc = nn.Linear(hidden_dim, pred_len)

    def forward(self, x):
        x_cnn = x.transpose(1, 2)
        x_cnn = self.cnn(x_cnn).transpose(1, 2)
        x_transformer = self.transformer_encoder(x_cnn)
        return self.fc(x_transformer[:, -1, :]).unsqueeze(-1)

# -------------------------- 数据加载 --------------------------
def load_public_data():
    data_path = "consumption.csv"
    df = pd.read_csv(data_path)
    
    df["DateTime"] = pd.to_datetime(df["DateTime"])  
    df.set_index("DateTime", inplace=True)  
    
    df_daily_mean = df.resample('D').mean() 
    # 新增：提取预测目标列，保留单变量时间序列
    target_col = "Zone 3  Power Consumption"
    df_daily_mean = df_daily_mean[[target_col]]  # 只保留目标列，形状变为(364,1)
    return df_daily_mean

# -------------------------- 探索性数据分析 --------------------------
def exploratory_data_analysis(df):
    print("\n" + "="*50)
    print("探索性数据分析")
    print("="*50)
    print(f"数据形状：{df.shape} | 时间范围：{df.index.min()} 至 {df.index.max()}")
    print("\n描述统计：")
    print(df.describe().round(2))
    
    # 时间序列图：修复多列数据问题，只取第一列（电力消耗列）进行绘图
    plt.figure(figsize=(15, 5))
    # 修改点1：使用df.iloc[:, 0]获取单列数据，避免多数据集
    plt.plot(df.index, df.iloc[:, 0].values, color='#2E86AB', linewidth=1.5)
    plt.title('日均电力消耗时间序列', fontsize=14, fontweight='bold')
    plt.xlabel('日期'), plt.ylabel('电力消耗 (kW)'), plt.grid(alpha=0.3)
    plt.tight_layout(), plt.savefig("eda_plots/time_series.png", dpi=300), plt.close()
    
    # 分布直方图：修复多列数据问题，只取第一列展平为一维数组
    plt.figure(figsize=(10, 6))
    # 修改点2：使用df.iloc[:, 0].values.flatten()获取单组数据
    plt.hist(df.iloc[:, 0].values.flatten(), bins=30, color='#F2A154', alpha=0.7, edgecolor='black')
    plt.title('电力消耗分布', fontsize=14, fontweight='bold')
    plt.xlabel('电力消耗 (kW)'), plt.ylabel('频数'), plt.grid(alpha=0.3)
    plt.tight_layout(), plt.savefig("eda_plots/distribution.png", dpi=300), plt.close()
    print("EDA可视化已保存至 eda_plots/ 目录")
    return df

# -------------------------- 数据预处理 --------------------------
def preprocess_data(df):
    # 一阶差分平稳化（此时df是(364,1)，差分后为(363,1)）
    diff_series = df.diff(1).dropna()  # 形状(363,1)
    # 标准化处理：直接对(363,1)的差分序列处理，无需reshape(-1,1)
    scaler = StandardScaler()
    diff_series_scaled = scaler.fit_transform(diff_series).flatten()  # 形状(363,)
    
    # 训练/测试集划分（基于真实的363条时序数据）
    train_size = int(0.8 * len(diff_series_scaled))  # 0.8×363≈290
    train_data = diff_series_scaled[:train_size]     # 约290条
    test_data = diff_series_scaled[train_size:]      # 约73条
    
    print(f"数据预处理完成：训练集{len(train_data)}条 | 测试集{len(test_data)}条")
    return train_data, test_data, scaler, diff_series.index

# -------------------------- 训练与评估 --------------------------
def train_eval(params, train_data, test_data, scaler, df_original, pred_len, epochs):
    try:
        train_dataset = TimeSeriesDataset(train_data, params['SEQ_LEN'], pred_len)
        test_dataset = TimeSeriesDataset(test_data, params['SEQ_LEN'], pred_len)
    except Exception as e:
        print(f"数据集构建失败：{e}")
        return float('inf'), float('inf'), None
    
    if len(train_dataset) < 10 or len(test_dataset) < 5:
        return float('inf'), float('inf'), None
    
    train_loader = DataLoader(train_dataset, batch_size=params['BATCH_SIZE'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=params['BATCH_SIZE'], shuffle=False)
    
    model = CNNTransformerModel(
        hidden_dim=params['hidden_dim'], num_heads=params['num_heads'],
        num_transformer_layers=params['num_transformer_layers'], kernel_size=params['kernel_size'],
        pred_len=pred_len
    ).to(DEVICE)
    
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=params['LEARNING_RATE'], weight_decay=1e-5)
    
    # 模型训练
    for _ in range(epochs):
        model.train()
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
            y_pred = model(x_batch)
            loss = criterion(y_pred, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
    
    # 预测函数
    def predict(model, loader):
        model.eval()
        preds, trues = [], []
        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                pred = model(x)
                preds.extend(pred.cpu().numpy().flatten())
                trues.extend(y.cpu().numpy().flatten())
        return np.array(preds), np.array(trues)
    
    # 逆变换函数
    def inverse_transform(pred_scaled, is_train=True):
        pred_diff = scaler.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
        start_idx = params['SEQ_LEN'] if is_train else len(train_data) + params['SEQ_LEN']
        original_vals = df_original.iloc[start_idx-1 : start_idx-1 + len(pred_diff)].values
        return original_vals + pred_diff
    
    # 测试集评估
    test_pred_scaled, test_true_scaled = predict(model, test_loader)
    test_pred = inverse_transform(test_pred_scaled, is_train=False)
    test_true = inverse_transform(test_true_scaled, is_train=False)
    
    test_mae = calculate_mae(test_true, test_pred)
    test_mape = calculate_mape(test_true, test_pred)
    return test_mae, test_mape, model

# -------------------------- 最优模型可视化 --------------------------
def visualize_best_model(best_params, best_model, train_data, test_data, scaler, df_original, pred_len):
    # 构建数据集
    best_train_dataset = TimeSeriesDataset(train_data, best_params['SEQ_LEN'], pred_len)
    best_test_dataset = TimeSeriesDataset(test_data, best_params['SEQ_LEN'], pred_len)
    best_train_loader = DataLoader(best_train_dataset, batch_size=best_params['BATCH_SIZE'], shuffle=False)
    best_test_loader = DataLoader(best_test_dataset, batch_size=best_params['BATCH_SIZE'], shuffle=False)
    
    # 预测函数
    def predict(model, loader):
        model.eval()
        preds, trues = [], []
        with torch.no_grad():
            for x, y in loader:
                x, y = x.to(DEVICE), y.to(DEVICE)
                pred = model(x)
                preds.extend(pred.cpu().numpy().flatten())
                trues.extend(y.cpu().numpy().flatten())
        return np.array(preds), np.array(trues)
    
    # 生成预测结果
    train_pred_scaled, train_true_scaled = predict(best_model, best_train_loader)
    test_pred_scaled, test_true_scaled = predict(best_model, best_test_loader)
    
    # 逆变换
    def inverse_transform_final(pred_scaled, is_train=True):
        pred_diff = scaler.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
        start_idx = best_params['SEQ_LEN'] if is_train else len(train_data) + best_params['SEQ_LEN']
        original_vals = df_original.iloc[start_idx-1 : start_idx-1 + len(pred_diff)].values
        return original_vals + pred_diff
    
    train_pred = inverse_transform_final(train_pred_scaled, is_train=True)
    train_true = inverse_transform_final(train_true_scaled, is_train=True)
    test_pred = inverse_transform_final(test_pred_scaled, is_train=False)
    test_true = inverse_transform_final(test_true_scaled, is_train=False)
    
    # 绘制图表
    fig, ax = plt.subplots(figsize=(15, 6))
    train_steps = range(len(train_true))
    test_steps = range(len(train_true), len(train_true) + len(test_true))
    
    ax.plot(train_steps, train_true, color='#2E86AB', linewidth=2.0, 
            label=f'训练集真实值 (MAE={calculate_mae(train_true, train_pred):.2f})', alpha=0.8)
    ax.plot(train_steps, train_pred, color='#F2A154', linewidth=1.8, label='训练集预测值', alpha=0.9)
    ax.plot(test_steps, test_true, color='#2E86AB', linewidth=2.5, 
            label=f'测试集真实值 (MAE={calculate_mae(test_true, test_pred):.2f})', alpha=0.8)
    ax.plot(test_steps, test_pred, color='#E63946', linewidth=2.2, label='测试集预测值', linestyle='--', alpha=0.9)
    ax.axvline(x=len(train_true)-1, color='black', linestyle=':', linewidth=2.0, label='训练/测试分割线')
    
    ax.set_title('电力消耗预测结果（CNN-Transformer）', fontsize=14, fontweight='bold')
    ax.set_xlabel('时间步'), ax.set_ylabel('电力消耗 (kW)'), ax.grid(alpha=0.3), ax.legend()
    plt.tight_layout(), plt.savefig("best_model/prediction_result.png", dpi=300), plt.close()
    print("预测结果可视化已保存")

# -------------------------- 主函数 --------------------------
def main():
    print("="*60)
    print("电力消耗预测项目")
    print("="*60)
    
    # 全局参数
    PRED_LEN = 1
    EPOCHS_BASE = 40
    
    # 超参数搜索空间
    param_grid = {
        'SEQ_LEN': [14, 28],
        'hidden_dim': [32, 64, 128],
        'num_heads': [2, 4, 8],
        'num_transformer_layers': [1, 2, 3],
        'kernel_size': [3, 5],
        'BATCH_SIZE': [16, 32],
        'LEARNING_RATE': [1e-4, 5e-4]
    }
    
    # 生成20组有效参数
    n_trials = 20
    random_params = []
    while len(random_params) < n_trials:
        params = {k: random.choice(v) for k, v in param_grid.items()}
        if params['hidden_dim'] % params['num_heads'] == 0:
            random_params.append(params)
    
    try:
        # 数据加载与EDA
        df_original = load_public_data()
        df_original = exploratory_data_analysis(df_original)
        
        # 数据预处理
        train_data, test_data, scaler, diff_index = preprocess_data(df_original)
        
        # 超参数搜索
        print("\n===== 开始20组超参数搜索 =====")
        results = []
        for i, params in enumerate(tqdm(random_params, desc="调参进度")):
            test_mae, test_mape, model = train_eval(params, train_data, test_data, scaler, df_original, PRED_LEN, EPOCHS_BASE)
            results.append({'params': params, 'test_mae': test_mae, 'test_mape': test_mape, 'model': model})
            print(f"第{i+1}组：MAE={test_mae:.6f} | MAPE={test_mape:.4f}%")
        
        # 筛选最优模型
        valid_results = [r for r in results if r['test_mae'] != float('inf')]
        if not valid_results:
            raise ValueError("无有效模型，请检查数据或参数范围")
        best_result = sorted(valid_results, key=lambda x: x['test_mae'])[0]
        best_params = best_result['params']
        best_model = best_result['model']
        best_mae = best_result['test_mae']
        best_mape = best_result['test_mape']
        
        # 最优模型可视化
        visualize_best_model(best_params, best_model, train_data, test_data, scaler, df_original, PRED_LEN)
        
        # 保存最优模型
        torch.save({
            'model_state_dict': best_model.state_dict(),
            'params': best_params,
            'scaler': scaler,
            'test_mae': best_mae,
            'test_mape': best_mape
        }, "best_model/best_model.pth")
        
        # 结果输出
        print("\n" + "="*60)
        print("最优模型结果汇总")
        print("="*60)
        print(f"最优参数：{best_params}")
        print(f"测试集MAE：{best_mae:.6f} kW | MAPE：{best_mape:.4f}%")
        print(f"\n输出文件：")
        print("- EDA可视化：eda_plots/")
        print("- 最优模型权重：best_model/best_model.pth")
        print("- 预测结果图：best_model/prediction_result.png")
    
    except Exception as e:
        print(f"\n运行失败：{str(e)}")
        raise

if __name__ == "__main__":
    main()


KeyboardInterrupt



## 多特征transformer+cnn模型

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import random
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

# 基础配置
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs("results", exist_ok=True)

# 评估指标计算
def calculate_mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def calculate_mape(y_true, y_pred, epsilon=1e-8):
    y_true = np.array(y_true) + epsilon
    y_pred = np.array(y_pred)
    return round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 4)

# 数据集类
class MultiFeatureTimeSeriesDataset(Dataset):
    def __init__(self, feat_data, target_data, seq_len, pred_len=1):
        self.feat_data = feat_data
        self.target_data = target_data
        self.seq_len = seq_len
        self.pred_len = pred_len
        
        if len(feat_data) != len(target_data):
            raise ValueError(f"特征与目标长度不一致（{len(feat_data)} vs {len(target_data)}）")
        if len(feat_data) < seq_len + pred_len:
            raise ValueError(f"数据长度不足（需≥{seq_len+pred_len}，实际{len(feat_data)}）")
        self.length = len(feat_data) - seq_len - pred_len + 1

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        if idx < 0 or idx >= self.length:
            raise IndexError(f"索引{idx}超出范围（0-{self.length-1}）")
        x = self.feat_data[idx:idx+self.seq_len]
        y = self.target_data[idx+self.seq_len:idx+self.seq_len+self.pred_len]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y.reshape(-1, 1), dtype=torch.float32)

# 模型类
class CNNTransformerModel(nn.Module):
    def __init__(self, input_dim=5, hidden_dim=64, num_heads=4, num_transformer_layers=2, kernel_size=3, pred_len=1):
        super().__init__()
        self.pred_len = pred_len
        
        if hidden_dim % num_heads != 0:
            raise ValueError(f"hidden_dim({hidden_dim})必须能被num_heads({num_heads})整除")
        if kernel_size % 2 == 0:
            raise ValueError(f"kernel_size({kernel_size})应为奇数")
        
        self.cnn = nn.Sequential(
            nn.Conv1d(input_dim, hidden_dim, kernel_size, padding=kernel_size//2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size, padding=kernel_size//2),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=hidden_dim, nhead=num_heads, dim_feedforward=hidden_dim*4,
                batch_first=True, norm_first=True, dropout=0.1
            ),
            num_layers=num_transformer_layers
        )
        
        self.fc = nn.Linear(hidden_dim, pred_len)

    def forward(self, x):
        x_cnn = x.transpose(1, 2)
        x_cnn = self.cnn(x_cnn).transpose(1, 2)
        x_trans = self.transformer_encoder(x_cnn)
        return self.fc(x_trans[:, -1, :]).unsqueeze(-1)

# 数据加载
def load_public_data():
    data_path = "consumption.csv"
    df = pd.read_csv(data_path)
    
    df["DateTime"] = pd.to_datetime(df["DateTime"])  
    df.set_index("DateTime", inplace=True)  
    
    df_daily_mean = df.resample('D').mean() 
    return df_daily_mean

# 探索性数据分析
def exploratory_data_analysis(df):
    print("\n" + "="*50)
    print("探索性数据分析")
    print("="*50)
    
    print(f"数据形状：{df.shape} | 时间范围：{df.index.min()} 至 {df.index.max()}")
    print("\n描述统计：")
    print(df.describe().round(2))
    
    if df.isnull().sum().sum() > 0:
        print("\n缺失值填充（前向+后向）：")
        df = df.fillna(method='ffill').fillna(method='bfill')
        print(f"填充后缺失值：{df.isnull().sum().sum()}")
    
    # 目标变量时间序列图
    plt.figure(figsize=(15, 4))
    plt.plot(df.index, df['Zone 3  Power Consumption'], color='#2E86AB', linewidth=1.5)
    plt.title('Zone 3 电力消耗时间序列', fontsize=12, fontweight='bold')
    plt.xlabel('日期'), plt.ylabel('电力消耗 (kW)'), plt.grid(alpha=0.3)
    plt.tight_layout(), plt.savefig("results/target_time_series.png", dpi=300), plt.close()
    
    # 特征相关性热力图
    plt.figure(figsize=(8, 6))
    corr = df.corr()
    plt.imshow(corr, cmap='RdBu_r', vmin=-1, vmax=1)
    plt.colorbar(label='相关系数'), plt.xticks(range(len(corr)), corr.columns, rotation=45, ha='right')
    plt.yticks(range(len(corr)), corr.columns), plt.title('特征-目标相关性热力图', fontsize=12, fontweight='bold')
    for i in range(len(corr)):
        for j in range(len(corr)):
            plt.text(j, i, f'{corr.iloc[i, j]:.2f}', ha='center', va='center', fontsize=8)
    plt.tight_layout(), plt.savefig("results/correlation_heatmap.png", dpi=300), plt.close()
    
    print("EDA可视化已保存至 results/ 目录")
    return df

# 数据预处理
def preprocess_data(df):
    FEATURE_COLS = ['Temperature', 'Humidity', 'Wind Speed', 'general diffuse flows', 'diffuse flows']
    TARGET_COL = 'Zone 3  Power Consumption'
    
    feat_data = df[FEATURE_COLS].values
    target_series = df[TARGET_COL].values
    
    # 目标序列一阶差分平稳化
    diff_target = np.diff(target_series, n=1)
    feat_data_aligned = feat_data[1:]
    
    # 标准化处理
    feat_scaler = StandardScaler()
    feat_data_scaled = feat_scaler.fit_transform(feat_data_aligned)
    
    target_scaler = StandardScaler()
    diff_target_scaled = target_scaler.fit_transform(diff_target.reshape(-1, 1)).flatten()
    
    # 划分训练/测试集
    train_size = int(0.8 * len(feat_data_scaled))
    train_feat, test_feat = feat_data_scaled[:train_size], feat_data_scaled[train_size:]
    train_target, test_target = diff_target_scaled[:train_size], diff_target_scaled[train_size:]
    
    print(f"数据预处理完成：训练集{len(train_feat)}条 | 测试集{len(test_feat)}条")
    return {
        'train_feat': train_feat, 'test_feat': test_feat,
        'train_target': train_target, 'test_target': test_target,
        'feat_scaler': feat_scaler, 'target_scaler': target_scaler,
        'original_target': target_series
    }

# 预测函数
def predict(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            pred = model(x)
            preds.extend(pred.cpu().numpy().flatten())
            trues.extend(y.cpu().numpy().flatten())
    return np.array(preds), np.array(trues)

# 逆变换函数
def inverse_transform(pred_scaled, target_scaler, original_target, seq_len, train_size, is_train=True):
    pred_diff = target_scaler.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
    start_idx = seq_len + 1
    if not is_train:
        start_idx += train_size
    prev_vals = original_target[start_idx-1 : start_idx-1 + len(pred_diff)]
    return prev_vals + pred_diff

# 训练与调参
def train_eval(prep_data, pred_len=1):
    print("\n" + "="*50)
    print("CNN-Transformer 训练与调参")
    print("="*50)
    
    # 超参数搜索空间
    param_grid = {
        'SEQ_LEN': [14, 28],
        'hidden_dim': [32, 64, 128],
        'num_heads': [2, 4, 8],
        'num_transformer_layers': [1, 2, 3],
        'kernel_size': [3, 5],
        'BATCH_SIZE': [16, 32],
        'LEARNING_RATE': [1e-4, 5e-4],
        'epochs': [40]
    }
    
    # 生成有效参数组合
    n_trials = 20
    valid_params = []
    while len(valid_params) < n_trials:
        params = {k: random.choice(v) for k, v in param_grid.items()}
        if params['hidden_dim'] % params['num_heads'] == 0:
            valid_params.append(params)
    
    # 搜索最优参数
    best_mae = float('inf')
    best_params = None
    best_model = None
    
    for i, params in enumerate(tqdm(valid_params, desc="调参进度")):
        try:
            # 构建数据集
            train_dataset = MultiFeatureTimeSeriesDataset(
                prep_data['train_feat'], prep_data['train_target'],
                seq_len=params['SEQ_LEN'], pred_len=pred_len
            )
            test_dataset = MultiFeatureTimeSeriesDataset(
                prep_data['test_feat'], prep_data['test_target'],
                seq_len=params['SEQ_LEN'], pred_len=pred_len
            )
            
            if len(train_dataset) < 10 or len(test_dataset) < 5:
                continue
            
            # 数据加载器
            train_loader = DataLoader(train_dataset, batch_size=params['BATCH_SIZE'], shuffle=False)
            test_loader = DataLoader(test_dataset, batch_size=params['BATCH_SIZE'], shuffle=False)
            
            # 初始化模型
            model = CNNTransformerModel(
                input_dim=5,
                hidden_dim=params['hidden_dim'],
                num_heads=params['num_heads'],
                num_transformer_layers=params['num_transformer_layers'],
                kernel_size=params['kernel_size'],
                pred_len=pred_len
            ).to(DEVICE)
            
            # 训练配置
            criterion = nn.L1Loss()
            optimizer = optim.Adam(model.parameters(), lr=params['LEARNING_RATE'], weight_decay=1e-5)
            epochs = params['epochs']
            
            # 模型训练
            model.train()
            for _ in range(epochs):
                for x_batch, y_batch in train_loader:
                    x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
                    y_pred = model(x_batch)
                    loss = criterion(y_pred, y_batch)
                    
                    optimizer.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    optimizer.step()
            
            # 测试集预测与评估
            test_pred_scaled, test_true_scaled = predict(model, test_loader)
            test_pred = inverse_transform(
                test_pred_scaled, prep_data['target_scaler'], prep_data['original_target'],
                params['SEQ_LEN'], len(prep_data['train_feat']), is_train=False
            )
            test_true = inverse_transform(
                test_true_scaled, prep_data['target_scaler'], prep_data['original_target'],
                params['SEQ_LEN'], len(prep_data['train_feat']), is_train=False
            )
            
            mae, mape = calculate_mae(test_true, test_pred), calculate_mape(test_true, test_pred)
            
            # 更新最优模型
            if mae < best_mae:
                best_mae = mae
                best_mape = mape
                best_params = params
                best_model = model
        
        except Exception as e:
            print(f"第{i+1}组参数训练失败：{str(e)}")
            continue
    
    # 保存最优模型
    torch.save({
        'model_state_dict': best_model.state_dict(),
        'params': best_params,
        'feat_scaler': prep_data['feat_scaler'],
        'target_scaler': prep_data['target_scaler'],
        'test_mae': best_mae,
        'test_mape': best_mape
    }, "results/best_model.pth")
    
    # 最优模型可视化
    train_dataset = MultiFeatureTimeSeriesDataset(
        prep_data['train_feat'], prep_data['train_target'],
        seq_len=best_params['SEQ_LEN'], pred_len=pred_len
    )
    train_loader = DataLoader(train_dataset, batch_size=best_params['BATCH_SIZE'], shuffle=False)
    test_loader = DataLoader(
        MultiFeatureTimeSeriesDataset(prep_data['test_feat'], prep_data['test_target'],
                                     seq_len=best_params['SEQ_LEN'], pred_len=pred_len),
        batch_size=best_params['BATCH_SIZE'], shuffle=False
    )
    
    # 训练集预测
    train_pred_scaled, train_true_scaled = predict(best_model, train_loader)
    train_pred = inverse_transform(
        train_pred_scaled, prep_data['target_scaler'], prep_data['original_target'],
        best_params['SEQ_LEN'], len(prep_data['train_feat']), is_train=True
    )
    train_true = inverse_transform(
        train_true_scaled, prep_data['target_scaler'], prep_data['original_target'],
        best_params['SEQ_LEN'], len(prep_data['train_feat']), is_train=True
    )
    
    # 计算训练集指标
    train_mae, train_mape = calculate_mae(train_true, train_pred), calculate_mape(train_true, train_pred)
    
    # 绘制可视化图
    fig, ax = plt.subplots(figsize=(15, 6))
    train_steps = range(len(train_true))
    test_steps = range(len(train_true), len(train_true) + len(test_true))
    
    ax.plot(train_steps, train_true, color='#2E86AB', linewidth=2.0, 
            label=f'训练集真实值 (MAE={train_mae:.2f} kW)', alpha=0.8)
    ax.plot(train_steps, train_pred, color='#F2A154', linewidth=1.8, label='训练集预测值', alpha=0.9)
    ax.plot(test_steps, test_true, color='#2E86AB', linewidth=2.5, 
            label=f'测试集真实值 (MAE={best_mae:.2f} kW)', alpha=0.8)
    ax.plot(test_steps, test_pred, color='#E63946', linewidth=2.2, label='测试集预测值', linestyle='--', alpha=0.9)
    ax.axvline(x=len(train_true)-1, color='black', linestyle=':', linewidth=2.0, label='训练/测试分割线')
    
    ax.set_title('Zone 3 电力消耗预测结果（5特征+CNN-Transformer）', fontsize=14, fontweight='bold')
    ax.set_xlabel('时间步'), ax.set_ylabel('电力消耗 (kW)'), ax.grid(alpha=0.3), ax.legend()
    plt.tight_layout(), plt.savefig("results/prediction_result.png", dpi=300), plt.close()
    
    # 输出最优结果
    print(f"\n最优结果：")
    print(f"最优参数：{best_params}")
    print(f"训练集MAE：{train_mae:.6f} | MAPE：{train_mape:.4f}%")
    print(f"测试集MAE：{best_mae:.6f} | MAPE：{best_mape:.4f}%")
    print("模型与可视化已保存至 results/ 目录")
    
    return {
        "params": best_params,
        "train_mae": train_mae,
        "test_mae": best_mae,
        "test_mape": best_mape
    }

# 主函数
def main():
    print("="*60)
    print("多特征电力消耗预测（Zone 3）")
    print("="*60)
    
    try:
        # 数据加载
        df = load_public_data()
        # 探索性数据分析
        df = exploratory_data_analysis(df)
        # 数据预处理
        prep_data = preprocess_data(df)
        # 模型训练与调参
        result = train_eval(prep_data, pred_len=1)
        
        print("\n" + "="*60)
        print("项目运行完成！")
        print("="*60)
        print("输出文件清单：")
        print("- EDA可视化：results/ 目录（时间序列图+相关性热力图）")
        print("- 最优模型权重：results/best_model.pth")
        print("- 预测结果图：results/prediction_result.png")
    
    except Exception as e:
        print(f"\n运行失败：{str(e)}")
        raise

if __name__ == "__main__":
    main()

多特征电力消耗预测（Zone 3）

探索性数据分析
数据形状：(364, 8) | 时间范围：2017-01-01 00:00:00 至 2017-12-30 00:00:00

描述统计：
       Temperature  Humidity  Wind Speed  general diffuse flows  \
count       364.00    364.00      364.00                 364.00   
mean         18.81     68.26        1.96                 182.70   
std           5.20     11.51        2.20                  86.16   
min           8.63     29.36        0.06                  26.44   
25%          14.20     60.03        0.08                 113.06   
50%          19.09     70.27        0.24                 166.30   
75%          22.87     76.94        4.91                 264.70   
max          32.43     89.59        4.93                 335.07   

       diffuse flows  Zone 1 Power Consumption  Zone 2  Power Consumption  \
count         364.00                    364.00                     364.00   
mean           75.03                  32344.97                   21042.51   
std            38.68                   2669.37                    27

调参进度: 100%|███████████████████████████████████| 1/1 [00:05<00:00,  5.73s/it]



最优结果：
最优参数：{'SEQ_LEN': 14, 'hidden_dim': 64, 'num_heads': 4, 'num_transformer_layers': 2, 'kernel_size': 5, 'BATCH_SIZE': 32, 'LEARNING_RATE': 0.0005, 'epochs': 40}
训练集MAE：333.852068 | MAPE：1.7299%
测试集MAE：481.362133 | MAPE：3.7866%
模型与可视化已保存至 results/ 目录

项目运行完成！
输出文件清单：
- EDA可视化：results/ 目录（时间序列图+相关性热力图）
- 最优模型权重：results/best_model.pth
- 预测结果图：results/prediction_result.png


## 多指标transformer+cnn联合模型

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import random
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

os.makedirs("multi_zone_best_model", exist_ok=True)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 评估指标函数
def calculate_mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def calculate_mape(y_true, y_pred, epsilon=1e-8):
    y_true = np.array(y_true) + epsilon
    y_pred = np.array(y_pred)
    return round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 4)

def calculate_multi_zone_metrics(true_dict, pred_dict):
    metrics = {}
    mae_list, mape_list = [], []
    for zone in ['z1', 'z2', 'z3']:
        mae = calculate_mae(true_dict[zone], pred_dict[zone])
        mape = calculate_mape(true_dict[zone], pred_dict[zone])
        metrics[f'{zone}_mae'], metrics[f'{zone}_mape'] = mae, mape
        mae_list.append(mae), mape_list.append(mape)
    metrics['avg_mae'], metrics['avg_mape'] = np.mean(mae_list), np.mean(mape_list)
    return metrics

# 多区域数据集类
class MultiZoneTimeSeriesDataset(Dataset):
    def __init__(self, multi_zone_data, seq_len, pred_len=1):
        self.multi_zone_data = multi_zone_data
        self.seq_len = seq_len
        self.pred_len = pred_len
        
        if len(multi_zone_data) < seq_len + pred_len:
            raise ValueError(f"数据长度不足（需≥{seq_len+pred_len}，实际{len(multi_zone_data)}）")
        self.length = len(multi_zone_data) - seq_len - pred_len + 1

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        if idx < 0 or idx >= self.length:
            raise IndexError(f"索引{idx}超出范围（0-{self.length-1}）")
        x = self.multi_zone_data[idx:idx+self.seq_len]
        y = self.multi_zone_data[idx+self.seq_len : idx+self.seq_len+self.pred_len]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# 多区域CNN-Transformer模型类
class MultiZoneCNNTransformer(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=64, num_heads=4, num_transformer_layers=2, kernel_size=3, pred_len=1):
        super().__init__()
        self.pred_len = pred_len
        
        if hidden_dim % num_heads != 0:
            raise ValueError(f"hidden_dim({hidden_dim})必须能被num_heads({num_heads})整除")
        
        self.cnn = nn.Sequential(
            nn.Conv1d(input_dim, hidden_dim, kernel_size, padding=kernel_size-1, dilation=1),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv1d(hidden_dim, hidden_dim, kernel_size, padding=2*(kernel_size-1), dilation=2),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=hidden_dim, nhead=num_heads, dim_feedforward=hidden_dim*4,
                batch_first=True, norm_first=True, dropout=0.1
            ),
            num_layers=num_transformer_layers
        )
        
        self.fc = nn.Linear(hidden_dim, pred_len * input_dim)

    def forward(self, x):
        x_cnn = x.transpose(1, 2)
        x_cnn = self.cnn(x_cnn).transpose(1, 2)
        x_trans = self.transformer_encoder(x_cnn)
        out = self.fc(x_trans[:, -1, :])
        return out.reshape(-1, self.pred_len, 3)

# 数据加载函数（修正后）
def load_data(data_path):
    if not os.path.exists(data_path):
        raise FileNotFoundError(f"数据文件不存在：{data_path}")
    
    # 修正1：使用原始日期列名"DateTime"，而非"date"
    df = pd.read_csv(data_path)
    # 手动解析日期列并设置为索引，提高兼容性
    df["DateTime"] = pd.to_datetime(df["DateTime"], errors='coerce')  # 错误日期转为NaT
    df = df.dropna(subset=["DateTime"])  # 删除无效日期行
    df.set_index("DateTime", inplace=True)  # 设置为DatetimeIndex
    
    # 修正2：确保索引是时间类型后再重采样
    if not isinstance(df.index, pd.DatetimeIndex):
        raise TypeError("索引未成功转为DatetimeIndex，请检查日期列格式")
    
    df_daily_mean = df.resample('D').mean().dropna()
    
    ZONE_COLS = {
        'z1': 'Zone 1 Power Consumption',
        'z2': 'Zone 2  Power Consumption',
        'z3': 'Zone 3  Power Consumption'
    }
    
    missing_cols = [col for col in ZONE_COLS.values() if col not in df_daily_mean.columns]
    if missing_cols:
        raise ValueError(f"数据缺失列：{missing_cols}")
    
    print(f"数据加载完成：日均值数据形状={df_daily_mean.shape} | 索引类型={type(df_daily_mean.index)}")
    return df_daily_mean, ZONE_COLS

# 数据预处理函数
def preprocess_data(df_daily_mean, ZONE_COLS):
    # 提取原始序列与差分
    raw_zones = {}
    diff_zones = {}
    for zone, col in ZONE_COLS.items():
        raw_zones[zone] = df_daily_mean[col].copy()
        diff_zones[zone] = raw_zones[zone].diff(1).dropna()
    
    # 对齐时间索引
    common_idx = diff_zones['z1'].index.intersection(diff_zones['z2'].index).intersection(diff_zones['z3'].index)
    for zone in diff_zones.keys():
        diff_zones[zone] = diff_zones[zone].loc[common_idx]
    
    # 标准化
    scalers = {}
    scaled_diff_zones = {}
    for zone in diff_zones.keys():
        scalers[zone] = StandardScaler()
        scaled_diff_zones[zone] = scalers[zone].fit_transform(diff_zones[zone].values.reshape(-1,1)).flatten()
    
    # 构建多区域矩阵
    multi_zone_data = np.column_stack([
        scaled_diff_zones['z1'], scaled_diff_zones['z2'], scaled_diff_zones['z3']
    ])
    
    # 划分训练/测试集
    train_size = int(0.8 * len(multi_zone_data))
    train_data = multi_zone_data[:train_size]
    test_data = multi_zone_data[train_size:]
    
    print(f"数据预处理完成：训练集{len(train_data)}条 | 测试集{len(test_data)}条")
    return train_data, test_data, scalers, raw_zones

# 预测与逆变换函数
def predict_multi_zone(model, loader):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            pred = model(x)
            preds.extend(pred.cpu().numpy()), trues.extend(y.cpu().numpy())
    return np.concatenate(preds, axis=0), np.concatenate(trues, axis=0)

def inverse_transform_multi_zone(scaled_diff, zone, scalers, raw_zones, seq_len, train_size, is_train=True):
    diff = scalers[zone].inverse_transform(scaled_diff.reshape(-1,1)).flatten()
    start_idx = seq_len if is_train else train_size + seq_len
    prev_raw_vals = raw_zones[zone].iloc[start_idx-1 : start_idx-1 + len(diff)].values
    return prev_raw_vals + diff

# 训练评估函数
def train_eval_multi_zone(params, train_data, test_data, PRED_LEN):
    try:
        train_dataset = MultiZoneTimeSeriesDataset(train_data, params['SEQ_LEN'], PRED_LEN)
        test_dataset = MultiZoneTimeSeriesDataset(test_data, params['SEQ_LEN'], PRED_LEN)
    except Exception as e:
        print(f"数据集构建失败：{e}")
        return float('inf'), {}, None
    
    if len(train_dataset) < 10 or len(test_dataset) < 5:
        return float('inf'), {}, None
    
    train_loader = DataLoader(train_dataset, batch_size=params['BATCH_SIZE'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=params['BATCH_SIZE'], shuffle=False)
    
    model = MultiZoneCNNTransformer(
        input_dim=3,
        hidden_dim=params['hidden_dim'],
        num_heads=params['num_heads'],
        num_transformer_layers=params['num_transformer_layers'],
        kernel_size=params['kernel_size'],
        pred_len=PRED_LEN
    ).to(DEVICE)
    
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=params['LEARNING_RATE'], weight_decay=1e-5)
    EPOCHS_BASE = 40
    
    # 训练
    for _ in range(EPOCHS_BASE):
        model.train()
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
            y_pred = model(x_batch)
            loss = criterion(y_pred, y_batch)
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
    
    # 测试集预测
    test_pred_scaled, test_true_scaled = predict_multi_zone(model, test_loader)
    return test_pred_scaled, test_true_scaled, model

# 可视化函数
def visualize_results(best_params, best_model, train_data, test_data, scalers, raw_zones, PRED_LEN):
    # 构建最优模型数据集
    best_train_dataset = MultiZoneTimeSeriesDataset(train_data, best_params['SEQ_LEN'], PRED_LEN)
    best_test_dataset = MultiZoneTimeSeriesDataset(test_data, best_params['SEQ_LEN'], PRED_LEN)
    best_train_loader = DataLoader(best_train_dataset, batch_size=best_params['BATCH_SIZE'], shuffle=False)
    best_test_loader = DataLoader(best_test_dataset, batch_size=best_params['BATCH_SIZE'], shuffle=False)
    
    # 预测
    train_pred_scaled, train_true_scaled = predict_multi_zone(best_model, best_train_loader)
    test_pred_scaled, test_true_scaled = predict_multi_zone(best_model, best_test_loader)
    
    # 逆变换
    zone_idx = {'z1':0, 'z2':1, 'z3':2}
    train_true, train_pred = {}, {}
    test_true, test_pred = {}, {}
    train_size = len(train_data)
    
    for zone, idx in zone_idx.items():
        train_true[zone] = inverse_transform_multi_zone(
            train_true_scaled[:, idx], zone, scalers, raw_zones, best_params['SEQ_LEN'], train_size, is_train=True
        )
        train_pred[zone] = inverse_transform_multi_zone(
            train_pred_scaled[:, idx], zone, scalers, raw_zones, best_params['SEQ_LEN'], train_size, is_train=True
        )
        test_true[zone] = inverse_transform_multi_zone(
            test_true_scaled[:, idx], zone, scalers, raw_zones, best_params['SEQ_LEN'], train_size, is_train=False
        )
        test_pred[zone] = inverse_transform_multi_zone(
            test_pred_scaled[:, idx], zone, scalers, raw_zones, best_params['SEQ_LEN'], train_size, is_train=False
        )
    
    # 计算指标
    train_metrics = calculate_multi_zone_metrics(train_true, train_pred)
    test_metrics = calculate_multi_zone_metrics(test_true, test_pred)
    
    # 绘制图表
    fig, axes = plt.subplots(3, 1, figsize=(18, 15), sharex=True)
    zones = ['z1', 'z2', 'z3']
    zone_names = ['Zone 1', 'Zone 2', 'Zone 3']
    colors = ['#2E86AB', '#F2A154', '#E63946']
    
    for i, (zone, zone_name) in enumerate(zip(zones, zone_names)):
        ax = axes[i]
        train_steps = range(len(train_true[zone]))
        test_steps = range(len(train_true[zone]), len(train_true[zone]) + len(test_true[zone]))
        
        ax.plot(train_steps, train_true[zone], color=colors[0], linewidth=2.0, 
                label=f'训练集真实值 (MAE={train_metrics[f"{zone}_mae"]:.2f} kW)', alpha=0.8)
        ax.plot(train_steps, train_pred[zone], color=colors[1], linewidth=1.8, 
                label='训练集预测值', linestyle='-', alpha=0.9)
        ax.plot(test_steps, test_true[zone], color=colors[0], linewidth=2.5, 
                label=f'测试集真实值 (MAE={test_metrics[f"{zone}_mae"]:.2f} kW)', alpha=0.8)
        ax.plot(test_steps, test_pred[zone], color=colors[2], linewidth=2.2, 
                label='测试集预测值', linestyle='--', alpha=0.9)
        ax.axvline(x=len(train_true[zone]), color='black', linestyle=':', linewidth=2.5, label='训练/测试集分割线')
        
        ax.set_title(f'{zone_name} 电力消耗 - 多区域联合预测结果', fontsize=14, fontweight='bold', pad=10)
        ax.set_ylabel('电力消耗 (kW)', fontsize=12)
        ax.tick_params(axis='x', labelsize=10), ax.tick_params(axis='y', labelsize=10)
        ax.grid(True, alpha=0.3, linestyle='--'), ax.legend(loc='upper right', fontsize=10)
    
    fig.suptitle('Zone 1/2/3 电力消耗多区域联合预测结果', fontsize=18, fontweight='bold', y=0.98)
    axes[-1].set_xlabel('时间步（按日期顺序）', fontsize=14)
    plt.tight_layout()
    plt.savefig("multi_zone_best_model/multi_zone_prediction.png", bbox_inches='tight', dpi=300)
    plt.show()
    
    return train_metrics, test_metrics

# 主函数（修正后）
def main():
    print("="*60)
    print("多区域电力消耗联合预测")
    print("="*60)
    
    # 配置参数
    DATA_PATH = "consumption.csv"
    PRED_LEN = 1
    
    # 超参数搜索空间（先定义，不使用train_data）
    param_grid = {
        'SEQ_LEN': [7, 14, 21, 28],
        'hidden_dim': [32, 64, 128, 256],
        'num_heads': [2, 4, 8, 16],
        'num_transformer_layers': [1, 2, 3, 4],
        'kernel_size': [3, 5, 7],
        'BATCH_SIZE': [8, 16, 32, 64],
        'LEARNING_RATE': [5e-5, 1e-4, 5e-4, 1e-3]
    }
    
    try:
        # 第一步：先完成数据加载与预处理（提前获取train_data）
        df_daily_mean, ZONE_COLS = load_data(DATA_PATH)
        train_data, test_data, scalers, raw_zones = preprocess_data(df_daily_mean, ZONE_COLS)
        
        # 第二步：再生成20组有效参数（此时train_data已存在，可正常引用）
        n_trials = 20
        random_params = []
        # 增加容错：避免因train_data过短导致条件无法满足
        seq_len_upper_limit = max(1, int(len(train_data)/10))
        print(f"\n生成超参数时，SEQ_LEN上限为：{seq_len_upper_limit}（基于train_data长度{len(train_data)}）")
        
        while len(random_params) < n_trials:
            params = {k: random.choice(v) for k, v in param_grid.items()}
            # 修正判断条件：使用已存在的train_data，同时增加seq_len合法性判断
            if (params['hidden_dim'] % params['num_heads'] == 0 and 
                params['SEQ_LEN'] <= seq_len_upper_limit and
                params['SEQ_LEN'] >= 1):
                random_params.append(params)
        
        # 第三步：超参数搜索（后续逻辑不变）
        print("\n===== 开始20组超参数随机搜索 =====")
        best_mae = float('inf')
        best_params = None
        best_model = None
        
        for i, params in enumerate(tqdm(random_params, desc="调参进度")):
            try:
                test_pred_scaled, test_true_scaled, model = train_eval_multi_zone(params, train_data, test_data, PRED_LEN)
                if test_pred_scaled.size == 0:
                    continue
                
                # 计算指标
                test_true = {}
                test_pred = {}
                zone_idx = {'z1':0, 'z2':1, 'z3':2}
                train_size = len(train_data)
                
                for zone, idx in zone_idx.items():
                    test_true[zone] = inverse_transform_multi_zone(
                        test_true_scaled[:, idx], zone, scalers, raw_zones, params['SEQ_LEN'], train_size, is_train=False
                    )
                    test_pred[zone] = inverse_transform_multi_zone(
                        test_pred_scaled[:, idx], zone, scalers, raw_zones, params['SEQ_LEN'], train_size, is_train=False
                    )
                
                metrics = calculate_multi_zone_metrics(test_true, test_pred)
                print(f"第{i+1}组：平均MAE={metrics['avg_mae']:.6f} | 平均MAPE={metrics['avg_mape']:.4f}%")
                
                if metrics['avg_mae'] < best_mae:
                    best_mae = metrics['avg_mae']
                    best_params = params
                    best_model = model
            
            except Exception as e:
                print(f"第{i+1}组参数训练失败：{str(e)}")
                continue
        
        # 最优模型结果（后续逻辑不变）
        print("\n" + "="*120)
        print("多区域联合预测最优模型结果汇总")
        print("="*120)
        if best_params is None:
            raise ValueError("未找到有效最优模型，请检查参数范围或数据质量")
        
        print(f"最优参数组合：")
        for k, v in best_params.items():
            print(f"  - {k}: {v}")
        
        # 可视化与保存
        train_metrics, test_metrics = visualize_results(best_params, best_model, train_data, test_data, scalers, raw_zones, PRED_LEN)
        
        print(f"\n📊 最终指标：")
        print(f"训练集 - 平均MAE：{train_metrics['avg_mae']:.6f} kW | 平均MAPE：{train_metrics['avg_mape']:.4f}%")
        print(f"测试集 - 平均MAE：{test_metrics['avg_mae']:.6f} kW | 平均MAPE：{test_metrics['avg_mape']:.4f}%")
        
        # 保存模型
        torch.save({
            'model_state_dict': best_model.state_dict(),
            'params': best_params,
            'scalers': scalers,
            'raw_zones': raw_zones,
            'train_metrics': train_metrics,
            'test_metrics': test_metrics,
            'zone_cols': ZONE_COLS
        }, "multi_zone_best_model/best_multi_zone_cnn_transformer_model.pth")
        
        print(f"\n💾 模型已保存至：multi_zone_best_model/best_multi_zone_cnn_transformer_model.pth")
        print(f"📈 可视化图已保存至：multi_zone_best_model/multi_zone_prediction.png")
    
    except Exception as e:
        print(f"\n运行失败：{str(e)}")
        raise

In [2]:
if __name__ == "__main__":
    main()

多区域电力消耗联合预测
数据加载完成：日均值数据形状=(364, 8) | 索引类型=<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
数据预处理完成：训练集290条 | 测试集73条

生成超参数时，SEQ_LEN上限为：29（基于train_data长度290）

===== 开始20组超参数随机搜索 =====


调参进度:   0%|                                          | 0/20 [00:02<?, ?it/s]

KeyboardInterrupt

